In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext wurlitzer

In [3]:
import pandas as pd
import numpy as np
import logging
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil
from pathlib import Path

logging.basicConfig(level = logging.DEBUG)

In [4]:
from ionbeam.sources import SmartCitizenKitSource
from ionbeam.core import parse_single_action, parse_config

config_file = Path("~/git/IonBeam-Deployment/config/ionbeam").expanduser()

action_yaml = """
class: SmartCitizenKitSource
copy_metadata_to_columns:
  - name: sensor_name
    key: sensor.name
"""

config, sck_source = parse_single_action(config_file, action_yaml, 
                        offline = True,
                        environment  = "local")

sck_source

DEBUG:ionbeam.core.config_parser.config_parser:Loaded global config...
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.data_path to /Users/math/git/IonBeam-Deployment/data
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_schema_path to /Users/math/git/IonBeam-Deployment/config/fdb/server/custom_schema
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.metkit_language_template to /Users/math/git/IonBeam-Deployment/config/metkit/language.yaml.template
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.secrets_file to /Users/math/git/IonBeam-Deployment/config/ionbeam/secrets.yaml
DEBUG:ionbeam.core.config_parser.config_parser:Checked repository source state: CodeSourceInfo(repo_status='Dirty', git_hash='731f9ea2c94aaba20321b0743dccb4c67902b91d')


ConfigMatchError: Invalid config yaml entry
        ???:?
        Determined the type to be 'SmartCitizenKitSource', check this is right.
        Extra keys: set()
        Missing keys: {'mappings'}
        Default keys (for reference): {'copy_metadata_to_columns', 'id', 'cache_directory', 'metadata', 'cache_version', 'use_cache', 'finish_after'}

        Values in the entry:
        {'class': 'SmartCitizenKitSource', 'copy_metadata_to_columns': [{'name': 'sensor_name', 'key': 'sensor.name'}]}
        

In [ ]:
from sqlalchemy.orm import Session
from ionbeam.metadata.db import *

init_db(config.globals)

In [ ]:
devices = sck_source.get_ICHANGE_devices()

In [8]:
devices = sck_source.get_ICHANGE_devices()
d = devices[0]
for sensor in d["data"]["sensors"]:
    readings = self.get_readings(chunk["device_id"], sensor["id"], chunk["start_date"], chunk["end_date"])


[]

In [7]:
from ionbeam.metadata import db
from shapely.geometry import Point

from ionbeam.sources.smart_citizen_kit.metadata import construct_sck_metadata(sck_source, device)

device = devices[0]
construct_sck_metadata(sck_source, device)


Station(id=None, external_id=28)


In [20]:
import json
device = devices[0]
id = device["id"]


with Session(sck_source.globals.sql_engine) as session:
    station = db.Station.find_by(session, external_id = str(id))
    print("Retrieved station: ", station)
    j = station.as_json()
    print(station)
    print(station.sensors[1])
    print(station.sensors[1].properties)

# print(json.dumps(j, indent = 4))

[autoreload of ionbeam.metadata.db failed: Traceback (most recent call last):
  File "/Users/math/micromamba/envs/ionbeam/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/math/micromamba/envs/ionbeam/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/math/micromamba/envs/ionbeam/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/math/micromamba/envs/ionbeam/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 335, in update_class
    if (old_obj == new_obj) is True:
        ^^^^^^^^^^^^^^^^^^
  File "/Users/math/micromamba/envs/ionbeam/lib/python3.11/site-packages/sqlalchemy/sql/operators.py", line 582, in __eq__
    return self.operate(eq, other)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/math/micromamba/envs/

Retrieved station:  Station(id=5bdf8734-8864-46c6-995e-3001a3810473, external_id='28')
Station(id=5bdf8734-8864-46c6-995e-3001a3810473, external_id='28')
Sensor(id=2, name='DHT22 - Humidity')
[Property(key=''Relative Humidity'', name='Relative Humidity', unit='%', description='Relative humidity is a measure of the amount of moisture in the air relative to the total amount of moisture the air can hold. For instance, if the relative humidity was 50%, then the air is only half saturated with moisture.')]


In [9]:
device["owner"]

{'id': 12,
 'uuid': '7ff5b84c-1f20-4ae4-a7f9-c588ebf51161',
 'username': 'sejarque',
 'url': '',
 'avatar': 'https://images.smartcitizen.me/s100/avatars/12-Yo.JPG',
 'profile_picture': 'https://api.smartcitizen.me/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBalVCIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--cb535c26e1ee4311479135fc16f1752d7bea5f9d/12-Yo.JPG',
 'location': {'city': 'MATADEPERA', 'country': None, 'country_code': None},
 'device_ids': [28]}

In [10]:
from shapely import Point, to_geojson

p = Point(1,2,3)
tuple(p.coords)

((1.0, 2.0, 3.0),)

In [11]:
with Session(sck_source.globals.sql_engine) as session:
    print(session.query(Station).all())
    # print(session.query(Station).filter_by(external_id = ).one_or_none())

[Station(id=5bdf8734-8864-46c6-995e-3001a3810473, external_id='28')]


In [12]:
{k : v  for k, v in devices[0].items() if k not in {"owner", "data"}}

{'id': 28,
 'uuid': '856e9670-03d1-481e-b9c5-6bf16474d993',
 'name': 'Manyi_08230',
 'description': 'Desde el 3/5/13: en exterior.\nEmplazamiento: Zona arbolada en limite de Parque Natural Sant Llorenç.\nSituado a unos 60m de la calle con trafico mas próxima.\n\nColocación sensor: desde ahora; 2/4/14 en caja kit.\nSituado bajo placa solar.\n\nAlimentación del sensor: Desde el 19/05/2013, conectado a placa solar de 12V-610mA Ref: ET-M53610 encarada a 182 Sur, con unos 45grados de inclinación. Desde 2/4/14, sin placa solar, alimentado permanente con alimentador.',
 'state': 'has_published',
 'system_tags': ['offline', 'outdoor'],
 'user_tags': ['Barcelona'],
 'last_reading_at': '2014-04-04T18:30:16Z',
 'created_at': '2013-04-24T22:17:56Z',
 'updated_at': '2024-06-27T17:03:14Z',
 'notify': {'stopped_publishing': False, 'low_battery': False},
 'device_token': '[FILTERED]',
 'postprocessing': None,
 'location': {'ip': None,
  'exposure': 'outdoor',
  'elevation': 680,
  'latitude': 41.61441

In [13]:
for msg in messages:
    print(msg.data.columns)
    print(msg.metadata.unstructured)

NameError: name 'messages' is not defined

In [ ]:

set(col for msg in messages for col in msg.data.columns)

In [ ]:
sck_source.get_sensor(5)

In [ ]:
from functools import cache

@cache
def sensor_ancestry(id):
    ancestry = []
    while True:
        try:
            s = sck_source.get_sensor(id)
        except:
            return ancestry[::-1]
        ancestry.append(s)
        if "parent_id" in s and s["parent_id"] is not None:
            id = s["parent_id"]
        else: break
    return ancestry[::-1]

def print_ancestry(id): print(" -> ".join(s["name"] for s in sensor_ancestry(id)))

print_ancestry(5)
print_ancestry(4)

In [ ]:
sck_source.get_sensor(5)

In [ ]:
sensor_map = {}

for d in devices:
    for i, s in enumerate(d["data"]["sensors"]):
        sensor_map[s["name"]] = dict(
            device_id = d["id"],
            sensor_id = s["id"],
            device_name = d["name"],
            sensor_name = s["name"],
            default_key = s["default_key"],
            ancestry = [s["name"] for s in sensor_ancestry(s["id"])],
            measurement = s["measurement"]["name"],
            unit = s["unit"]
        )
            
        # print("name", )
        # print("default_key", )
        # print_ancestry(s["id"])
        # print("Measurement", s["measurement"]["name"])
        # print()
        # print(s["measurement"]["description"])
        # print()
    # print(devices[0]["data"]["sensors"][0])

sensor_map

In [ ]:
by_measurement = {}
for s in sensor_map.values():
    by_measurement[s["measurement"]] = by_measurement.get(s["measurement"], []) + [s,]

by_measurement

In [ ]:
s = sck_source.get_sensor(7)
s

In [ ]:
s = 135
while True:
    sensor = sck_source.get_sensor(s)
    print(sensor)
    print()
    if "parent_id" in sensor and sensor["parent_id"] is not None and sensor["parent_id"] != s:
            s = sensor["parent_id"]
    else:
        break
        
        
